In [ ]:
'''
Мы использовали модель BERT (Bidirectional Encoder Representations from Transformers) для решения задачи классификации текста по настроению. Модель BERT основана на принципе трансформеров, который позволяет модели эффективно обрабатывать последовательности данных. Основные компоненты модели BERT включают в себя:

Трансформеры: BERT состоит из множества блоков трансформеров. Каждый блок состоит из множества слоев, включая слой внимания и слой нормализации. Слои внимания позволяют модели обрабатывать взаимодействия между словами во всех позициях последовательности, а слои нормализации помогают ускорить обучение и улучшить стабильность модели.

Механизм самовнимания: Этот механизм позволяет модели сосредоточиться на наиболее важных словах в предложении при решении задач обработки естественного языка. BERT использует механизм внимания для эффективного анализа контекста в тексте.

Многоуровневая предварительная обработка: Перед тем как входить в модель BERT, текстовые данные проходят через несколько уровней предварительной обработки, включая токенизацию, встраивание токенов и добавление специальных токенов для представления начала и конца последовательности.

Многофункциональность: BERT способен решать различные задачи в области обработки естественного языка, включая классификацию, вопросно-ответную систему, именованное сущностьное извлечение и т. д.
'''
import pandas as pd
from transformers import BertTokenizer
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from torch.nn.utils import clip_grad_norm_
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm
import numpy as np
import math

In [ ]:
# Чтение обучающего набора данных из CSV файла
df = pd.read_csv("drive/MyDrive/train.csv")

# Определение уникальных меток классов
label_names = df['sentiment'].unique()

# Разделение данных на обучающий и валидационный наборы
train_df = df[:100000]
valid_df = df[100000:120000]

# Использование предварительно обученного BERT для токенизации текста
PRETRAINED_LM = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_LM, do_lower_case=True)

In [ ]:
# Функция для кодирования текста в токены и создания масок внимания
def encode(docs):
    encoded_dict = tokenizer.batch_encode_plus(docs, add_special_tokens=True, max_length=128, padding='max_length',
                                               return_attention_mask=True, truncation=True, return_tensors='pt')
    input_ids = encoded_dict['input_ids']
    attention_masks = encoded_dict['attention_mask']
    return input_ids, attention_masks

In [ ]:
train_input_ids, train_att_masks = encode(train_df['text'].values.tolist())
valid_input_ids, valid_att_masks = encode(valid_df['text'].values.tolist())

# Создание тензоров для меток классов
train_y = torch.LongTensor(train_df['sentiment'].values.tolist())
valid_y = torch.LongTensor(valid_df['sentiment'].values.tolist())

BATCH_SIZE = 16

# Создание DataLoader для обучающего и валидационного наборов данных
train_dataset = TensorDataset(train_input_ids, train_att_masks, train_y)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

valid_dataset = TensorDataset(valid_input_ids, valid_att_masks, valid_y)
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=BATCH_SIZE)

N_labels = len(train_df.sentiment.unique())

In [ ]:
# Инициализация модели BERT для классификации последовательностей
model = BertForSequenceClassification.from_pretrained(PRETRAINED_LM,
                                                      num_labels=N_labels,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Установка устройства для обучения на GPU, если доступно
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.cuda()

In [ ]:
EPOCHS = 30
LEARNING_RATE = 2e-6

# Определение оптимизатора и планировщика для обучения модели
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(train_dataloader) * EPOCHS)

In [ ]:
# Цикл обучения модели
train_loss_per_epoch = []
val_loss_per_epoch = []

for epoch_num in range(EPOCHS):
    print('Epoch: ', epoch_num + 1)

    model.train()
    train_loss = 0
    for step_num, batch_data in enumerate(tqdm(train_dataloader, desc='Training')):
        input_ids, att_mask, labels = [data.to(device) for data in batch_data]
        output = model(input_ids=input_ids, attention_mask=att_mask, labels=labels)

        loss = output.loss
        train_loss += loss.item()

        model.zero_grad()
        loss.backward()
        del loss

        clip_grad_norm_(parameters=model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

    train_loss_per_epoch.append(train_loss / (step_num + 1))

    model.eval()
    valid_loss = 0
    valid_pred = []
    with torch.no_grad():
        for step_num_e, batch_data in enumerate(tqdm(valid_dataloader, desc='Validation')):
            input_ids, att_mask, labels = [data.to(device) for data in batch_data]
            output = model(input_ids=input_ids, attention_mask=att_mask, labels=labels)

            loss = output.loss
            valid_loss += loss.item()

            valid_pred.append(np.argmax(output.logits.cpu().detach().numpy(), axis=-1))

    val_loss_per_epoch.append(valid_loss / (step_num_e + 1))
    valid_pred = np.concatenate(valid_pred)

    print("{0}/{1} train loss: {2} ".format(step_num + 1, math.ceil(len(train_df) / BATCH_SIZE),
                                            train_loss / (step_num + 1)))
    print("{0}/{1} val loss: {2} ".format(step_num_e + 1, math.ceil(len(valid_df) / BATCH_SIZE),
                                          valid_loss / (step_num_e + 1)))

Epoch:  1


Training:   0%|          | 0/63 [00:00<?, ?it/s]

Validation:   0%|          | 0/13 [00:00<?, ?it/s]

63/63 train loss: 1.0543790459632874 
13/13 val loss: 1.0516383877167335 


In [ ]:
# Сохранение модели и использование обученной модели для предсказаний
torch.save(model.state_dict(), "myTrainedModel")

model.eval()

test_predictions = []

true_labels = []

with torch.no_grad():
    for batch_data in tqdm(valid_dataloader, desc='Testing'):
        input_ids, att_mask, labels = [data.to(device) for data in batch_data]
        output = model(input_ids=input_ids, attention_mask=att_mask)

        predictions = torch.argmax(output.logits, dim=1).cpu().detach().numpy()

        test_predictions.extend(predictions)
        true_labels.extend(labels.cpu().detach().numpy())

f1 = f1_score(true_labels, test_predictions, average='weighted')
print("F1 Score:", f1)

Testing:   0%|          | 0/13 [00:00<?, ?it/s]

F1 Score: 0.28986301369863016
